In [3]:
#Script developed by Damian Dalle Nogare at the Human Technopole Image Analysis Facility, National Facility for Data Handling and Analysis
#Liscenced under the BSD-3 license

#import packages
import skimage
import numpy as np
import os
import math
import pandas as pd
from imaris_ims_file_reader.ims import ims as load_ims

In [38]:
#define functions

def get_labels_in_tracked_image(tracked_labs):
    '''Takes a tracked file and returns all of the labels present at any frame
    of the tracked image'''
    labels = []
    for i in range(0, tracked_labs.shape[0]):
        props = skimage.measure.regionprops(tracked_labs[i,:,:,:])
        for prop in props:
            if prop['label'] not in labels:
                labels.append(prop['label'])
    return labels

def get_first_transcription(raw_data):
    '''takes a raw data image and returns the first frame where the 
    trancription channel has non-zero values.
    Returns -1 if no non-zero pixel values ever exist in the RNA channel'''
    rna = raw_data[:,5,:,:,:]
    for i in range(0, raw_data.shape[0]):
        if np.amax(rna[i,:,:,:]) > 0:
            return i
    return -1

def find_first_transcrition(raw_data, tracked_labs, label, distance = 10):
    '''takes a tracked image, a raw data image, and a label value and returns the frame
    at which transcription begins (within a 'distance' (default 10) pixel radius of the track centroid).
    Returns -1 if no transcription is detected
    '''
    first_transcription = get_first_transcription(raw_data)
    for i in range(first_transcription, tracked_labs.shape[0]):
        track_props = skimage.measure.regionprops(tracked_labs[i,:,:,:])
        rna = raw_data[i,5,:,:,:]
        rna[rna > 0] = 1
        labs_transcription = skimage.measure.label(rna)
        props_transcription = skimage.measure.regionprops(labs_transcription)

        for track_prop in track_props:
            if track_prop['label'] == label:
                if len(props_transcription) > 0:
                    for trans_prop in props_transcription:
                        if math.dist(track_prop['centroid'], trans_prop['centroid']) < distance:
                            return i
    return -1
    
def get_labels_in_slice(lab_slice):
    '''takes a single labelled timepoint and returns the labels present'''
    labels_in_slice = []
    props = skimage.measure.regionprops(lab_slice)
    for prop in props:
        labels_in_slice.append(prop['label'])
    return labels_in_slice

def calculate_border_distances(im, tracked_filename, frame, label):
    '''calculates the mean and standard deviation between the centroid and all the border pixels
    for a single max-projected convex hull image'''
    prop = skimage.measure.regionprops(im)
    distance_array = []
    hull_edge = np.zeros(im.shape, dtype=np.uint8)
    for coord in prop[0]['coords']:
        if is_edge_pixel(im, coord[0], coord[1]):
            distance_array.append(round(math.dist(prop[0]['centroid'], coord),2))
            hull_edge[coord[0], coord[1]] = 255
    mean = np.mean(distance_array)
    stdev = np.std(distance_array)
    return mean, stdev, distance_array, hull_edge

def calculate_radial_distances_on_convex_hull(tracked_labs, label, tracked_filename):
    '''takes a tracked, label image mean/stdev distance between centroid and the edge pixels
    for a given label'''

    chull_dist_mean = []
    chull_dist_stdev =[]
    chull_dist_array = []
    max_projection = np.max(tracked_labs, axis=1)
    chull_stack = np.zeros(max_projection.shape)

    for i in range(0, max_projection.shape[0]):

        im_single = max_projection[i,:,:]
        labels_in_slice = get_labels_in_slice(im_single)

        if label not in labels_in_slice:
            chull_dist_mean.append(np.NaN)
            chull_dist_stdev.append(np.NaN)
            chull_dist_array.append(np.NaN)
            
        else:
            im_copy = np.zeros(im_single.shape)
            im_copy[im_single == label] = 255
            chull = skimage.morphology.convex_hull_image(im_copy)
            
            mean_distances_euc, stdev_distances_euc, distance_array, hull_edge = calculate_border_distances(chull.astype(np.uint8), tracked_filename, i, label)
            chull_dist_mean.append(mean_distances_euc)
            chull_dist_stdev.append(stdev_distances_euc)
            chull_dist_array.append(distance_array)
            chull_stack[i] = hull_edge


    return chull_dist_mean, chull_dist_stdev, chull_dist_array

def Calculate_ZYX_and_spot_numbers(tracked_labs, label):
    '''for a tracked_lab image and a label, returns an array of the z, y and x coordinates, as well as the number of spots per frame'''
    z_pos = []
    y_pos = []
    x_pos = []
    num_spots_array = []

    single_label = np.zeros(tracked_labs.shape, dtype = np.uint8)
    single_label[tracked_labs == label] = 1
    
    
    for i in range(0, single_label.shape[0]):
        labs, numspots = skimage.measure.label(single_label[i,:,:,:], return_num=True)
        props = skimage.measure.regionprops(labs)
        if len(props) > 0:
            z_pos.append(props[0]['centroid'][0])
            y_pos.append(props[0]['centroid'][1])
            x_pos.append(props[0]['centroid'][2])
            num_spots_array.append(numspots)

        else:
            z_pos.append(np.NaN)
            y_pos.append(np.NaN)
            x_pos.append(np.NaN)
            num_spots_array.append(np.NaN)

    return z_pos, y_pos, x_pos, num_spots_array

def is_edge_pixel(im, y, x):
    '''for a given pixel coordinate (y, x) in an image (im), calculates whether it is on the border of a mask (non-zero pixel adjacent'''
    val = im[y, x]
    if im[y, x-1] == 0:
        return True
    if im[y, x+1] == 0:
        return True
    if im[y-1, x] == 0:
        return True
    if im[y+1, x] == 0:
        return True
    return False

def process_single_image(tracked_file, raw_file):
    
    tracked_filename = tracked_file.split(os.path.sep)[-1]  
    print(tracked_filename)
    df = pd.DataFrame()
    all_transcription_starts = []
    all_labels_at_transcription_starts = []
    labels_at_transcription_starts = []
    transcription_starts = []
    print(raw_file)
    print(raw_file[-4:])
    if raw_file[-4:] == '.ims':
        raw_data = load_ims(raw_file)
    elif raw_file[-4:] == '.tif':
        raw_data = skimage.io.imread(raw_file)

    tracked_labs = skimage.io.imread(tracked_file)

    labels_present = get_labels_in_tracked_image(tracked_labs)

    print(raw_data.shape)
    print(tracked_labs.shape)

    for label in labels_present:

        labels_at_transcription_starts.append(label)
        first_trans = find_first_transcrition(raw_data, tracked_labs, label)
        transcription_starts.append(first_trans)
 
    for i, label in enumerate(labels_at_transcription_starts):
        if transcription_starts[i] >= 0:
            transcription_start_array = []
            for tp in range(0, tracked_labs.shape[0]):
                    if tp < transcription_starts[i]:
                        transcription_start_array.append(0)
                    else:
                        transcription_start_array.append(1)

            all_transcription_starts.append(transcription_starts[i])
            all_labels_at_transcription_starts.append(label)
            mean, stdev, dist_array = calculate_radial_distances_on_convex_hull(tracked_labs, label, tracked_filename)
            z_pos, y_pos, x_pos, num_spots_array = Calculate_ZYX_and_spot_numbers(tracked_labs, label)


            df['timepoint'] = np.arange(0,20)
            df['label_' + str(label) + '_transcription'] = transcription_start_array
            df['label_' + str(label) + '_mean_edge_distace'] = mean
            df['label_' + str(label) + '_std_edge_distace'] = stdev
            df['label_' + str(label) + '_distance_array'] = dist_array
            df['label_' + str(label) + 'number_of_spots'] = num_spots_array
            df['label_' + str(label) + 'z_pos'] = z_pos
            df['label_' + str(label) + 'y_pos'] = y_pos
            df['label_' + str(label) + 'x_pos'] = x_pos
            

    return df

In [39]:

#pass in a raw data image, a tracked image, and returns a dataframe with the calculated data

raw_image = 'test_data/test_raw.ims'
tracked_image = 'test_data/test_tracked.tif'

df = process_single_image(tracked_image, raw_image)

test_tracked.tif
test_data/test_raw.ims
.ims
Opening readonly file: test_data/test_raw.ims 

(20, 6, 56, 228, 166)
(20, 56, 228, 166)
Closing file: test_data/test_raw.ims 

